<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/template/template.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Clasificación supervisada Máxima verosimilitud
Script trabajado por Francisco Albornoz (2021-04-30 15:00:24 ) / Última modificación / Análisis y clasificación supervisada región de Aysen

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [1]:
# Installs geemap package
# import subprocess

# try:
#     import geemap
# except ImportError:
#     print("Installing geemap ...")
#     subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

In [2]:
import os
import ee
import geemap
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Crear un mapa interactivo 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Escogemos la imagen landsat 

In [4]:
centroid = ee.Geometry.Point([-72, -44])

landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(centroid) \
    .filterDate('2020-01-01', '2021-03-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

landsat_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(centroid, 8)
Map.addLayer(landsat, landsat_vis, "Landsat-8")


In [5]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")

landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")

Map

Map(center=[-44, -72], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Importar rasters locales 

In [6]:
%pwd

'/mnt/c/users/Usuario/Documents/GitHub/VitoAlbornoz/GEE'

In [7]:
# verificar en el sistema del pc cual es la ruta
# os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/') 

In [8]:
# verificar en el sistema del pc cual es la ruta
out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/'), 'Data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

raster_aysen = os.path.join(out_dir, 'landcover_2018_aysen.tif')

In [9]:
out_dir

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data'

In [10]:
raster_aysen

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif'

In [11]:
Map.add_raster(raster_aysen, colormap= 'terrain', layer_name='Landcover 2018')
Map

Map(center=[-44, -72], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [12]:
landcover = ee.Image("users/franciscoalbornoz/landcover_sur")
landcover

In [13]:
raster = rasterio.open(out_dir+'/landcover_2018_aysen.tif')

In [14]:
raster.bounds

BoundingBox(left=-73.169644655, bottom=-46.394042378, right=-70.719691266, top=-43.705745621)

In [15]:
raster.crs

CRS.from_epsg(4326)

In [16]:
raster

<open DatasetReader name='/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif' mode='r'>

## Obteniendo los valores para el Training Datasets

In [17]:
# points = landcover.sample(**{
# #     'region': landsat.geometry(),
#     'scale': 30,
#     'numPixels': 20000,
#     'seed': 0,
#     'geometries': True  # Set this to False to ignore geometries
# })

# Map.addLayer(points, {}, 'training', False)
# Map

In [18]:
work_dir = os.path.expanduser('../Data/shapes/')

merged_shp = os.path.join(work_dir, 'roi_h.shp')
val_shp= os.path.join(work_dir, 'validacion.shp')
merged_original =os.path.join(work_dir,'roi_merged.shp')

In [19]:
merged = geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [20]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')

In [21]:
mergedoriginal =geemap.shp_to_ee(merged_original)
Map.addLayer(mergedoriginal, {}, 'Áreas de entrenamiento Original')

In [22]:
dataset = mosaico.sampleRegions(**{
    'collection': merged,
    'properties': ['IDH'],
    'scale' : 30 ,
})

In [23]:
dataset1 = mosaico.sampleRegions(**{
    'collection': mergedoriginal,
    'properties': ['ID'],
    'scale': 30,
})

In [24]:
datasetval = mosaico.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [25]:
print(dataset1.size().getInfo())

14254


In [26]:
print(datasetval.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'ID': 3, 'b1': 0.01979999989271164, 'b2': 0.017999999225139618, 'b3': 0.010499999858438969, 'b4': 0.01730000041425228, 'b5': 0.00989999994635582, 'b6': 0.0071000000461936, 'b7': 0.24460433423519135, 'b8': -0.019829994067549706, 'b9': 0.019325504079461098}}


### Dividiendo en muestras y datos de validación

In [27]:
# Guardamos las bandas
band = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']
# Esta propiedad guarda las clases de landcover
# label = 'b1'
label = 'IDH'
label1 ='ID'
# Overlay the points on the imagery to get training.
# sample = landsat.select(bands).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# # Adds a column of deterministic pseudorandom numbers. 
# sample = sample.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [28]:
# training.first().getInfo()

In [29]:
# validation.first().getInfo()

## Entrenando el modelo 

In [30]:
classifier = ee.Classifier.minimumDistance("mahalanobis").train(dataset1, label1, bands)

In [31]:
classifier1 = ee.Classifier.minimumDistance("euclidean").train(dataset1, label1, bands)

In [32]:
classifier100 = ee.Classifier.minimumDistance("cosine").train(dataset1, label1, bands)

### Clasificando la imagen

In [33]:
# Clasificando con las mismas bandas de entrenamiento
result1 = mosaico.select(bands).classify(classifier)
result2 = mosaico.select(bands).classify(classifier1)
result3 = mosaico.select(bands).classify(classifier100)
# # Display the clusters with random colors.
# Map.addLayer(result.randomVisualizer(), {}, 'classificado')
# Map

In [34]:
# result.getInfo()

### Categorizando según las clases 

In [35]:
# landcover.getInfo()

In [36]:
class_values = [1,2,3,4,5,6,7,8,9,10,11]
class_values1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
class_palette = ['0c40f1', #1 agua*
 'f0ff00', #2 cultivo*
 'f99504', #3 estepa*
 '17d4c6', #4 humedal*
 '24ff00', #5 matorral abierto*
 '2de50f', #6 matorral arborescente*
 'ccba7d', #7 plantaciones*
 'dec61a', #8 praderas*
 '28d909', #9 primario*
 '4e8c44', #10 secundario*
 '685205', #11 suelo desnudo*
]
class_palette1 = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [37]:
# class_values = [1,
#  2,
#  3,
#  4,
#  5,
#  6,
#  7,
#  8,
#  9,
#  10,
#  11,
#  12,
#  13,
#  14,
#  15,
#  16]

In [38]:
# class_palette = ['22c129',
#  'f0ff00',
#  '0c40f1',
#  '685205',
#  '0d4404',
#  'e82d13',
#  'f99504',
#  '17d4c6',
#  '1c6330',
#  '24ff00',
#  '2de50f',
#  'ccba7d',
#  'e3e3c2',
#  '28d909',
#  '03c2ff',
#  '4e8c44',
# ]


In [39]:
# clases = ["Bosque Adulto",       #1
#            "Renoval",             #2
#            "Matorrales",          #3
#            "Praderas",            #4
#            "Agropecuario",        #5
#            "Agua",                #6
#            "Humedales",           #7 
#            "Suelo Desnudo",       #8
#            "Plantación Forestal", #9
#            "Incendio",            #10
#            "Urbano",              #11
#            "Estepa",              #12
#            "Estepa abierta",      #13
#            "Glaciares-Nieve",     #14
#            "Vegetación de altura",#15
#            "No clasificado"]     #16

In [40]:
landcover1 = result1.set('classification_class_values', class_values)
landcover1 = landcover1.set('classification_class_palette', class_palette)

In [41]:
landcover2 = result2.set('classification_class_values', class_values1)
landcover2 = landcover2.set('classification_class_palette', class_palette1)

In [42]:
landcover3 = result3.set('classification_class_values', class_values1)
landcover3 = landcover3.set('classification_class_palette', class_palette1)

In [43]:
Map.addLayer(landcover1, {}, 'Land cover clasificado MD Mahalanobis' )

In [44]:
Map.addLayer(landcover2, {}, 'Land cover clasificado MD Euclidean' )

In [45]:
Map.addLayer(landcover3, {}, 'Land cover clasificado MD Cosine' )
Map

Map(center=[-45.0498939995, -71.9446679605], controls=(WidgetControl(options=['position'], widget=HBox(childre…

### Visualizando el resultado


In [46]:
# print('Change layer opacity:')
# cluster_layer = Map.layers[-1]
# cluster_layer.interact(opacity=(0, 1, 0.1))

### Agregando título al mapa

In [47]:
# Map.add_legend('Clasificación supervisada Random Forest')
# Map

### Evaluación de presición / Accuracy assessment

In [48]:
train_accuracy = classifier1.confusionMatrix()

In [49]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 393, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 39, 0, 190, 0, 0],
 [0, 3, 647, 0, 0, 1, 0, 0, 0, 120, 0, 0, 0, 4, 0, 0, 31, 0, 0],
 [0, 0, 0, 26, 442, 0, 0, 37, 1233, 0, 0, 0, 0, 0, 0, 1277, 0, 5, 0],
 [0, 0, 0, 0, 1329, 0, 3, 284, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 52, 5, 0, 0, 0, 4, 39, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 48, 724, 0, 0, 0, 7, 144, 8, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 7, 0, 0, 614, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 5, 0, 0, 0, 0, 43, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 84, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 218, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 59, 1, 0, 0, 0, 3, 95, 4, 0, 0, 0, 0, 0, 0],
 [0, 780, 2, 0, 0, 55, 19, 0, 0, 0, 5, 376, 1337, 0, 425, 0, 99, 0, 0],
 [0, 0, 7, 0, 0, 1, 0, 28, 0, 29, 8, 0, 0, 143, 0, 0, 0, 0, 42],
 [0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 115, 0, 639, 0, 3, 0, 0],
 [0, 0, 0, 46, 0, 0, 0, 0, 0, 0, 0, 0

In [50]:
train_accuracy.accuracy().getInfo()

0.5394976848603901

In [51]:
train_accuracy.kappa().getInfo()

0.5081454528980315

In [52]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.6288],
 [0.8027295285359801],
 [0.008609271523178808],
 [0.8188539741219963],
 [0.52],
 [0.7776584317937701],
 [0.9887278582930756],
 [0.8775510204081632],
 [0.9767441860465116],
 [0.9819819819819819],
 [0.5864197530864198],
 [0.4315687540348612],
 [0.5542635658914729],
 [0.8181818181818182],
 [0.7880184331797235],
 [0.7132352941176471],
 [0.9870967741935484],
 [0.6548780487804878]]

In [53]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.28437047756874095,
  0.9803030303030303,
  0.33766233766233766,
  0.7474690663667042,
  0.20634920634920634,
  0.9614873837981408,
  0.6115537848605578,
  0.03369905956112853,
  0.24068767908309455,
  0.8104089219330854,
  0.14285714285714285,
  0.9113837764144512,
  0.6355555555555555,
  0.5793291024478694,
  0.11801242236024845,
  0.6002475247524752,
  0.9683544303797469,
  0.927461139896373]]

Random Forest 100

In [54]:
train_accuracy1 = classifier100.confusionMatrix()

In [55]:
train_accuracy1.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 558, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 7, 0, 53, 0, 0],
 [0, 2, 729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 47, 0, 0, 28, 0, 0],
 [0, 0, 0, 132, 0, 3, 7, 0, 763, 0, 11, 0, 0, 10, 0, 1178, 0, 916, 0],
 [0, 0, 0, 0, 1308, 1, 4, 274, 0, 0, 11, 0, 0, 2, 0, 0, 0, 0, 23],
 [0, 0, 0, 0, 0, 60, 7, 0, 0, 0, 3, 30, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 8, 770, 0, 0, 0, 1, 118, 0, 0, 34, 0, 0, 0, 0],
 [0, 0, 0, 0, 3, 0, 0, 618, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 0, 0, 0, 38, 0, 0, 0, 0, 0, 0, 2, 0, 7, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 86, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 216, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 59, 9, 0, 0, 0, 6, 81, 0, 0, 7, 0, 0, 0, 0],
 [0, 10, 1, 0, 0, 26, 0, 0, 0, 0, 1, 145, 940, 0, 1168, 0, 807, 0, 0],
 [0, 0, 16, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 192, 0, 0, 0, 0, 34],
 [0, 42, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 80, 0, 640, 0, 0, 0, 0],
 [0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 

In [56]:
train_accuracy1.accuracy().getInfo()

0.5413918899957907

In [57]:
train_accuracy1.kappa().getInfo()

0.5130163232857051

In [58]:
train_accuracy1.producersAccuracy().getInfo()

[[0],
 [0.8928],
 [0.9044665012406948],
 [0.04370860927152318],
 [0.8059149722735675],
 [0.6],
 [0.8270676691729323],
 [0.9951690821256038],
 [0.7755102040816326],
 [1],
 [0.972972972972973],
 [0.5],
 [0.30342156229825695],
 [0.7441860465116279],
 [0.8194622279129321],
 [0.9493087557603687],
 [0.7294117647058823],
 [1],
 [0.6]]

In [59]:
train_accuracy1.consumersAccuracy().getInfo()

[[0,
  0.7117346938775511,
  0.9370179948586118,
  0.9103448275862069,
  0.9977116704805492,
  0.37037037037037035,
  0.9661229611041405,
  0.6248736097067745,
  0.04744069912609238,
  0.7049180327868853,
  0.7741935483870968,
  0.20610687022900764,
  0.9117361784675073,
  0.46943765281173594,
  0.3448275862068966,
  0.14862914862914864,
  0.3583815028901734,
  0.1437847866419295,
  0.8961748633879781]]

#### Set de datos de validación

In [60]:
validated = datasetval.classify(classifier1)

In [61]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'ID': 3,
  'b1': 0.01979999989271164,
  'b2': 0.017999999225139618,
  'b3': 0.010499999858438969,
  'b4': 0.01730000041425228,
  'b5': 0.00989999994635582,
  'b6': 0.0071000000461936,
  'b7': 0.24460433423519135,
  'b8': -0.019829994067549706,
  'b9': 0.019325504079461098,
  'classification': 8}}

In [62]:
test_accuracy = validated.errorMatrix('ID', 'classification')

In [63]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 28, 0, 0, 5, 0, 0, 0, 5, 0, 0, 0, 3, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 24, 0, 0, 12, 8, 0, 0, 0, 2, 0, 5, 0, 0, 0, 0],
 [0, 0, 0, 0, 581, 0, 0, 41, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 20, 0, 0, 250, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 68],
 [0, 0, 0, 0, 0, 0, 7, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 9, 13, 0, 0, 15, 0, 13, 0, 1, 26, 43, 7, 14, 0, 0, 0, 0, 1],
 [0, 41, 13, 0, 0, 6, 5, 0, 0, 0, 2, 99, 58, 2, 39, 0, 2, 0, 0],
 [0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 13, 0, 0, 0, 0],
 [0, 0, 99, 0, 0, 28, 0, 9, 0, 23, 7, 9, 0, 80, 0, 0, 0, 0, 36],
 [0, 457, 20, 0, 8, 13, 0, 0, 0, 0, 0, 33, 109, 0, 373, 0, 87, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [

In [64]:
test_accuracy.accuracy().getInfo()

0.4956409428479173

In [65]:
test_accuracy.kappa().getInfo()

0.42670230997710484

In [66]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [1],
 [0.6829268292682927],
 [0],
 [0.9236883942766295],
 [0.75],
 [0],
 [0.7396449704142012],
 [0],
 [0.5],
 [0.18309859154929578],
 [0.3707865168539326],
 [0.4375],
 [0.27491408934707906],
 [0.3390909090909091],
 [0],
 [0.4695121951219512],
 [0],
 [0]]

In [67]:
test_accuracy.consumersAccuracy().getInfo()

[[0,
  0.005319148936170213,
  0.16,
  0,
  0.8829787234042553,
  0.04285714285714286,
  0,
  0.7668711656441718,
  0,
  0.03333333333333333,
  0.7428571428571429,
  0.5351351351351351,
  0.07035175879396985,
  0.7476635514018691,
  0.8233995584988962,
  0,
  0.463855421686747,
  0,
  0]]

RF 100 Validación

In [68]:
validated1 = datasetval.classify(classifier100)

In [69]:
validated1.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'ID': 3,
  'b1': 0.01979999989271164,
  'b2': 0.017999999225139618,
  'b3': 0.010499999858438969,
  'b4': 0.01730000041425228,
  'b5': 0.00989999994635582,
  'b6': 0.0071000000461936,
  'b7': 0.24460433423519135,
  'b8': -0.019829994067549706,
  'b9': 0.019325504079461098,
  'classification': 17}}

In [70]:
test_accuracy1 = validated1.errorMatrix('ID', 'classification')

In [71]:
test_accuracy1.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 37, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 44, 0],
 [0, 0, 0, 0, 564, 0, 0, 56, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 26, 0, 0, 285, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27],
 [0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 16, 14, 0, 0, 24, 0, 7, 0, 0, 23, 30, 0, 14, 5, 0, 1, 0, 8],
 [0, 80, 13, 0, 0, 21, 0, 0, 0, 0, 2, 100, 15, 2, 27, 0, 7, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 13, 0, 10, 0, 0],
 [0, 0, 117, 0, 0, 7, 0, 12, 0, 1, 47, 7, 0, 70, 0, 0, 0, 0, 30],
 [0, 589, 20, 0, 8, 4, 0, 0, 0, 0, 0, 38, 107, 0, 281, 0, 53, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 

In [72]:
test_accuracy1.accuracy().getInfo()

0.4743299967710688

In [73]:
test_accuracy1.kappa().getInfo()

0.4112986079350377

In [74]:
test_accuracy1.producersAccuracy().getInfo()

[[0],
 [1],
 [0.9024390243902439],
 [0],
 [0.8966613672496025],
 [0.75],
 [0],
 [0.8431952662721893],
 [0],
 [1],
 [0.1619718309859155],
 [0.37453183520599254],
 [0.28125],
 [0.24054982817869416],
 [0.25545454545454543],
 [0],
 [0.5609756097560976],
 [0],
 [0]]

In [75]:
test_accuracy1.consumersAccuracy().getInfo()

[[0,
  0.004149377593360996,
  0.18137254901960784,
  0,
  0.9052969502407705,
  0.05084745762711865,
  0,
  0.7916666666666666,
  0,
  0.3333333333333333,
  0.2911392405063291,
  0.5681818181818182,
  0.06293706293706294,
  0.8045977011494253,
  0.8005698005698005,
  0,
  0.5644171779141104,
  0,
  0]]

### Descargando la matrix

In [76]:
# import csv
# import os

# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/'), 'Desktop')
# training_csv = os.path.join(out_dir, 'train_accuracy.csv')
# testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

# with open(training_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(train_accuracy.getInfo())
    
# with open(testing_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(test_accuracy.getInfo())

### Exportando 

In [77]:
# import os
# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/'), 'Desktop')
# out_file = os.path.join(out_dir, 'landcover.tif')

In [78]:
# geemap.ee_export_image(landcover, filename=out_file, scale=900)


In [79]:
# geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)